## Building GPT from Scatch

In [1]:
# downloading toy dataset 
import os
import requests

WORK_dir = '/n/projects/kc2819/projects/ChotaLLM'
DATA_dir =  os.path.join(WORK_dir, 'data')
SRC_dir = os.path.join(WORK_dir, 'src')

input_file_path = os.path.join(DATA_dir,'shakespeare', 'input.txt')
if not os.path.exists(input_file_path):
    data_url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
    with open(input_file_path, 'w', encoding='utf-8') as f:
        f.write(requests.get(data_url).text)

with open(input_file_path, 'r', encoding='utf-8') as f:
    text = f.read()
    print(text[:1000])
    print('text length:', len(text))


First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



## simple index based tokenization

* get unique characters --> vocabulary 
* create a simple tokenizer to convert a string to number with vocabulary.
* Todo that useing simple encoder decoder kind fo character mapping.

In [40]:
class simpleTokenizer:

    '''
    Simple tokenizer that encode decode character level tokenization.
    Alternatives could be google sentencepiece, huggingface tokenizers, openai Tiktoken library.
    
    '''

    def __init__(self,text):
        self.unique_chars = sorted(list(set(text)))
        self.vocab_size = len(self.unique_chars)
        # strign to index mapping
        self.str_to_idx = {chr: idx for idx,chr in enumerate(self.unique_chars)}
        # index to string mapping
        self.idx_to_str = {idx: chr for idx,chr in enumerate(self.unique_chars)}

        # adding padding and unknown token
        self.str_to_idx['<PAD>'] = self.vocab_size
        self.idx_to_str[self.vocab_size] = '<PAD>'
        self.unique_chars.append('<PAD>')

        self.str_to_idx['<UNK>'] = self.vocab_size + 1
        self.idx_to_str[self.vocab_size + 1] = '<UNK>'
        self.unique_chars.append('<UNK>')

        self.vocab_size += 2

    def encode(self, text):
        '''
        this function takes a string and look up in the string to index mapping, and return the list of indices.
        '''
 
        return [self.str_to_idx[chr] if chr in self.unique_chars else self.str_to_idx['<UNK>'] for chr in text]
        
        
    def decode(self, indices):
        '''
        this function takes a list of indices and look up in the index to string mapping, and return the string.
        '''
        return ''.join([self.idx_to_str[idx] for idx in indices]) 
    
    def get_vocab_size(self):
        return self.vocab_size
    
    def get_unique_chars(self):
        return self.unique_chars


In [41]:
tokenizer = simpleTokenizer(text)
print('vocab size:', tokenizer.get_vocab_size())
print('unique chars:', ''.join(tokenizer.get_unique_chars()))

vocab size: 67
unique chars: 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz<PAD><UNK>


In [42]:
# test cast
encoded_text = tokenizer.encode('hello, IM Bhuvan, asdflasdf &&*&&)))())')
decoded_text = tokenizer.decode(encoded_text)
print('encoded text:', encoded_text)
print('decoded text:', decoded_text)

encoded text: [46, 43, 50, 50, 53, 6, 1, 21, 25, 1, 14, 46, 59, 60, 39, 52, 6, 1, 39, 57, 42, 44, 50, 39, 57, 42, 44, 1, 4, 4, 66, 4, 4, 66, 66, 66, 66, 66, 66]
decoded text: hello, IM Bhuvan, asdflasdf &&<UNK>&&<UNK><UNK><UNK><UNK><UNK><UNK>


### tokenizing data


In [47]:
import torch

In [51]:
data = torch.tensor(tokenizer.encode(text),dtype = torch.long)
print('encoded data shape, type \n' ,data.shape,data.dtype)
print( 'encoded data: \n', data[:100])
print('decoded data: \n', tokenizer.decode(data[:100].tolist()))

encoded data shape, type 
 torch.Size([1115394]) torch.int64
encoded data: 
 tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])
decoded data: 
 First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


### data splits

In [52]:
n = int(0.9 * len(data))
train_data, val_data = data[:n], data[n:]

block size or context length is a small chunk of data in a a sequencial fashion. When we feed this block of string into a transformer, with respect to string{1 -> n} n+1th character is the target for the tranfomer. From the lecture chunk_size or block_size = 8.

In [53]:
block_size = 8
x = train_data[:block_size]  # block with 0 -> block_size for the training input 
y = train_data[1:block_size+1] # block with 1 -> block_size+1 for the training target, shifted by 1 from the input

for i in range(block_size):
    context = x[:i+1]
    target = y[i]
    print(f'for context:{context},target is {target} \n')
    


KeyError: tensor(47)